In [1]:
import optuna
import joblib
import datetime

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': 200}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_max=360)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        subsample = trial.suggest_float("subsample", 0.01, 1)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'colsample_bytree': colsample_bytree, 'num_boost_round': num_boost_round}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_min=360)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    
    return result

In [4]:
data_dir = "../data/Sulfur_Models/heating/heating_all"
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 1 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = True
study_name = "study_XGBoost_20220708-19.pkl"

In [5]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

Best trial until now:
 Value:  5.524612415643093
 Params: 
    learning_rate: 0.7303137624182325
    subsample: 0.7460404902385005
    colsample_bytree: 0.8555578185452548


In [6]:
study.optimize(objective, n_trials=250)

[23:17:51] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:32.26539
[20]	test-rmse:8.18279
[28]	test-rmse:8.63249
Split #0, This Result: 8.8979, Average Result: 8.8979
[23:18:20] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:28.44063
[20]	test-rmse:9.57957
[40]	test-rmse:9.62026
[45]	test-rmse:9.61449
Split #1, This Result: 9.6149, Average Result: 9.2564

[I 2022-07-08 23:20:20,260] Trial 9 finished with value: 7.156570678081183 and parameters: {'learning_rate': 0.31184190124088107, 'subsample': 0.6107729263035355, 'colsample_bytree': 0.18476491504401746}. Best is trial 2 with value: 5.524612415643093.


Split #4, This Result: 5.0902, Average Result: 7.1566
[23:20:35] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:23.40025
[20]	test-rmse:9.61493
[23]	test-rmse:9.42959
Split #0, This Result: 9.5413, Average Result: 9.5413
[23:21:01] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:21.76374
[20]	test-rmse:9.04227
[40]	test-rmse:9.66384
[42]	test-rmse:9.66759

[I 2022-07-08 23:22:32,953] Trial 10 finished with value: 7.477252481847981 and parameters: {'learning_rate': 0.530736353445202, 'subsample': 0.5955359879866121, 'colsample_bytree': 0.5024268524729334}. Best is trial 2 with value: 5.524612415643093.


Split #4, This Result: 5.9517, Average Result: 7.4773
[23:22:46] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:44.22303
[20]	test-rmse:28.71423
[40]	test-rmse:19.63630
[60]	test-rmse:14.82174
[80]	test-rmse:12.35599
[100]	test-rmse:11.02347
[120]	test-rmse:10.09757
[140]	test-rmse:9.62545
[160]	test-rmse:9.35514
[180]	test-rmse:9.16467
[200]	test-rmse:8.92783
[220]	test-rmse:8.77007
[240]	test-rmse:8.61751
[260]	test-rmse:8.58850
[280]	test-rmse:8.41936
[300]	test-rmse:8.38986
[320]	test-rmse:8.37398
[340]	test-rmse:8.36272
[360]	test-rmse:8.34879
[380]	test-rmse:8.20878
[400]	test-rmse:8.17097
[414]	test-rmse:8.18536
Split #0, This Result: 8.1856, Average Result: 8.18

...

[I 2022-07-09 07:29:12,133] Trial 139 finished with value: 4.722344863444509 and parameters: {'learning_rate': 0.46124250324792426, 'subsample': 0.14713002237097456, 'colsample_bytree': 0.8127547175856622}. Best is trial 137 with value: 4.027244048489939.


Split #4, This Result: 3.6338, Average Result: 4.7223
[07:29:24] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:44.95664
[20]	test-rmse:37.42224
[40]	test-rmse:31.34906
[60]	test-rmse:26.17444
[80]	test-rmse:21.95126
[100]	test-rmse:18.82295
[120]	test-rmse:16.11039
[140]	test-rmse:14.05706
[160]	test-rmse:12.56549
[180]	test-rmse:11.41693
[200]	test-rmse:10.56036
[220]	test-rmse:9.86866
[240]	test-rmse:9.34681
[260]	test-rmse:8.94748
[280]	test-rmse:8.59541
[300]	test-rmse:8.33373
[320]	test-rmse:8.13163
[340]	test-rmse:8.01081
[360]	test-rmse:7.92084
[380]	test-rmse:7.85652
[400]	test-rmse:7.81478
[420]	test-rmse:7.75002
[440]	test-rmse:7.71873
[460]	test-rmse:7.69261

[I 2022-07-09 07:43:46,879] Trial 140 finished with value: 4.507178167524972 and parameters: {'learning_rate': 0.01064461951138608, 'subsample': 0.13679218651705616, 'colsample_bytree': 0.8770281366104874}. Best is trial 137 with value: 4.027244048489939.


Split #4, This Result: 3.1810, Average Result: 4.5072
[07:43:59] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:42.31253
[20]	test-rmse:13.84933
[40]	test-rmse:8.60634
[60]	test-rmse:7.88448
[80]	test-rmse:7.74835
[100]	test-rmse:7.24961
[120]	test-rmse:6.62319
[140]	test-rmse:6.20741
[160]	test-rmse:5.87849
[180]	test-rmse:5.69889
[200]	test-rmse:5.68887
[220]	test-rmse:5.58911
[240]	test-rmse:5.54277
[260]	test-rmse:5.54155
[263]	test-rmse:5.54306
Split #0, This Result: 5.5431, Average Result: 5.5431
[07:45:02] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by lang


KeyboardInterrupt



In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621